Import all necessary librarys

In [4]:
import numpy as np
import sys, os
import time
import matplotlib.pyplot as plt
import sqlite3 as sql
import scipy as sp

%matplotlib qt

os.chdir(r'd:\Documenten\Documents\GitHub\Active-Vibration-Stabilization\Meetfiles\data\2023-04-15')
print(os.getcwd())

d:\Documenten\Documents\GitHub\Active-Vibration-Stabilization\Meetfiles\data\2023-04-15


In [5]:
fit = lambda f, a , b, c, d : a*f**3 + b*f**2 + c*f**1 +d

In [6]:
transferFunction = np.load(f'(2023-04-15) Full Transfer Function.npy')
print(transferFunction)
# transferFunction = np.delete(transferFunction, [12,19],0)
# transferFunction = np.delete(transferFunction, [2,5,12,14,15,17,24,25,29,41,42,43,44, 46,47,49],0)
# transferFunction = np.delete(transferFunction, [11,15,18,22,23,25,29,30,39,49,50],0)

frequencies = np.abs(transferFunction[:,:,0])
amplitudes = np.abs(transferFunction[:,:,3])

fitFrequencies = np.linspace(np.min(frequencies),np.max(frequencies), len(frequencies[0])*1000)
fits = [[] for _ in range(len(frequencies[0]))]

for i in range(len(frequencies[0])):
    (fa,fb,fc,fd),_ = sp.optimize.curve_fit(fit, frequencies[:,i], 20*np.log10(amplitudes[:,i]))
    fits[i] = np.append(fits[i],fit(fitFrequencies, fa,fb,fc,fd))
fits = np.array(fits)
fitFrequencies = np.array([fitFrequencies for _ in range(len(frequencies[0]))])
print(f'{fits}')
print(sp.optimize.curve_fit(fit, frequencies[:,i], 20*np.log10(amplitudes[:,i])))



phases = np.rad2deg(np.angle(transferFunction[:,:,3]))
 
# phases[[0,2,3],4] += 360
# phases[11:,[5,1]] -= 360
# phases[[13,14,15,16,17,18,19,20,27,28,30,32],4] += 360
# for i, ph in enumerate(phases):
#     for j, p in enumerate(ph):
#         if i > 11:
#             phases[i,j] =  p -360 if p >0 else p


# amp = 20*np.log10(np.average(amplitudes[8:],0)*np.ones_like(frequencies))
# _3db = amp-6


[[ 5.00247795e-01+0.00000000e+00j  7.41157277e-04+1.05121314e-03j
  -2.81011410e-01-5.34612269e-02j -2.81752568e-01-5.45124400e-02j]
 [ 5.50272575e-01+0.00000000e+00j -4.34662064e-04-8.12125580e-04j
  -1.19850960e-01+2.28373094e-02j -1.19416298e-01+2.36494350e-02j]
 [ 6.00297354e-01+0.00000000e+00j -8.61117469e-04+7.69434122e-04j
   1.91520195e-01+4.02829637e-02j  1.92381312e-01+3.95135296e-02j]
 [ 6.50322134e-01+0.00000000e+00j  6.91164851e-04+1.71845307e-04j
   2.90031534e-01-6.51896079e-02j  2.89340369e-01-6.53614532e-02j]
 [ 7.00346913e-01+0.00000000e+00j -1.61951365e-04-4.05216441e-06j
   1.43810998e-01-1.10738801e-01j  1.43972949e-01-1.10734749e-01j]
 [ 7.50371693e-01+0.00000000e+00j -1.05032315e-04+4.88403914e-04j
  -2.09684356e-01-1.52059496e-01j -2.09579324e-01-1.52547900e-01j]
 [ 8.00396472e-01+0.00000000e+00j  4.86858789e-04-2.82334656e-04j
  -2.02247574e-01+3.04268000e-01j -2.02734433e-01+3.04550335e-01j]
 [ 8.50421252e-01+0.00000000e+00j -1.24295942e-04-3.34374027e-04j
  -

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
fig2, (bode_amplitude, bode_phase) = plt.subplots(2,1)
fig2_manager = plt.get_current_fig_manager()
fig2_manager.set_window_title('bodeplot south Z')
fig2_manager.window.showMaximized()

bode_amplitude.sharex(bode_phase)

bode_amplitude.set_title('South Z actuator Bode plot')
bode_amplitude.set_xscale('log'); bode_amplitude.set_yscale('linear')
bode_amplitude.set_ylabel('Magnitude $(dB)$')
bode_amplitude.set_xlabel('Frequency $(Hz)$')
bode_amplitude.grid(True,'both')
bode_amplitude.set_xlim(0.5*0.95,3.0*1.05)

bode_phase.set_ylabel('fase $arg(H(s))$  $[deg]$')
bode_phase.set_xlabel('Frequency $[Hz]$')
bode_phase.grid(True, 'both')
bode_phase.set_ylim(-450,200)

colors = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22']
labels = 'x y z rx ry rz'.split()
labelsfit = ['Fitted x','Fitted y','Fitted z','Fitted rx','Fitted ry','Fitted rz']
# labels ='topX topY topZ northX northY northZ eastX eastY eastZ'.split()
# labelsfit = ['Fitted topX','Fitted topY','Fitted topZ','Fitted northX','Fitted northY','Fitted northZ','Fitted eastX','Fitted eastY','Fitted eastZ']

for f,a,p,ff,af,c,l,lf in zip(np.transpose(frequencies), np.transpose(20*np.log10(amplitudes)), np.transpose(phases), fitFrequencies, fits, colors, labels, labelsfit):
    _3db = (np.max(af)-6)*np.ones_like(ff)
    _3dbi = np.where((af - _3db) >= 0)[0]
    
    i = np.where((_3dbi[1:] - _3dbi[:-1]) != 1)[0]
    _3dbi = _3dbi[:i[0]+1] if len(i) > 0 else _3dbi
    f_min = ff[_3dbi[0]]
    f_max = ff[_3dbi[-1]]
    f_res = ff[np.argmax(af)]
    Q = f_res / np.abs(f_max-f_min)

    print(f'''
{l}
    f_res: {f_res:.2f}
    f_min: {f_min:.2f}
    f_max: {f_max:.2f}
    q_fac: {Q:.2f}
    ''')

    bode_amplitude.plot(ff,af,color = c,linestyle='-.',label=lf)
    bode_amplitude.plot(f, a, color = c, marker='o', linestyle='none', label=l)
    bode_amplitude.plot(ff[_3dbi], _3db[_3dbi],linestyle='none', color = c)
    bode_phase.plot(f, p, c,label=l)
    
    
    
bode_amplitude.legend()
bode_phase.legend()


# bode_amplitude.plot(frequencies, amp, '-', frequencies, _3db, ':')

plt.pause(0.002)
fig2.tight_layout()


x
    f_res: 1.40
    f_min: 0.89
    f_max: 2.19
    q_fac: 1.09
    

y
    f_res: 1.46
    f_min: 0.89
    f_max: 2.57
    q_fac: 0.87
    

z
    f_res: 1.65
    f_min: 1.06
    f_max: 2.90
    q_fac: 0.90
    

rx
    f_res: 1.53
    f_min: 0.94
    f_max: 2.90
    q_fac: 0.78
    

ry
    f_res: 1.68
    f_min: 1.08
    f_max: 2.90
    q_fac: 0.92
    

rz
    f_res: 1.44
    f_min: 0.89
    f_max: 2.34
    q_fac: 0.99
    
